# Data Wrangling I

Data Wrangling é um processo manual para transformar dados brutos em um formato adequado para análise.
ETL é um processo automatizado projetado para integrar, limpar e preencher dados em um repositório, normalmente um data warehouse. Embora a organização de dados seja exploratória e iterativa, o ETL é sistemático e definido.

## Coleta de Dados

A partir da lista de ativos da API Alpha Vantage em conjunto com a biblioteca Yahoo Finance, vamos criar uma nova base de dados com as ações, informando os dividendos dos últimos 5 anos. 


In [1]:
import pandas as pd

df = pd.read_csv('listing_status.csv', header=0)
df.head(10)

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
5,AACT,Ares Acquisition Corporation II - Class A,NYSE,Stock,2023-06-12,NaN,Active
6,AACT-U,Ares Acquisition Corporation II - Units (1 Ord...,NYSE,Stock,2023-04-21,NaN,Active
7,AACT-WS,Ares Acquisition Corporation II - Warrants (01...,NYSE,Stock,2023-06-12,NaN,Active
8,AADI,Aadi Bioscience Inc,NASDAQ,Stock,2017-08-08,NaN,Active
9,AADR,ADVISORSHARES DORSEY WRIGHT ADR ETF,NASDAQ,ETF,2010-07-21,NaN,Active


In [2]:
#! pip install yfinance
import yfinance as yf

def get_finance_data_name(symbol):
    try:
        return yf.Ticker(symbol).info['shortName']
    except:
        return f''

## Limpeza e Transformação dos Dados

Faremos um processo mais minucioso de limpeza dos dados, pois sabemos que há muitas inconsistências de tipos:

In [3]:
df.dtypes

symbol            object
name              object
exchange          object
assetType         object
ipoDate           object
delistingDate    float64
status            object
dtype: object

E também de dados ausentes:

In [4]:
df.isna().sum()

symbol               1
name                34
exchange             0
assetType            0
ipoDate              0
delistingDate    11628
status               0
dtype: int64

In [5]:
df.loc[df.name.isna()]

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
456,AMEH,NaN,NASDAQ,Stock,2024-02-26,NaN,Active
487,AMRS,NaN,NASDAQ,Stock,2023-08-18,NaN,Active
896,AVRO,NaN,NASDAQ,Stock,2024-06-21,NaN,Active
2113,CLVS,NaN,NASDAQ,Stock,2023-01-03,NaN,Active
2419,CTEST,NaN,NYSE,Stock,2019-07-25,NaN,Active
2632,DEC,NaN,NYSE,Stock,2023-12-18,NaN,Active
2676,DFFN,NaN,NASDAQ,Stock,2023-08-17,NaN,Active
4135,FWP,NaN,NASDAQ,Stock,2022-12-27,NaN,Active
6994,MTEST,NaN,NYSE,Stock,2019-10-09,NaN,Active
7401,NTEST-G,NaN,NYSE,Stock,2019-07-17,NaN,Active


In [6]:
df.loc[df.symbol.isna()]

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
7073,NaN,Nano Labs Ltd,NASDAQ,Stock,2022-07-12,NaN,Active


Vamos corrigir esses tipos para lidar melhor com essas informações:

In [7]:
df['ipoDate'] = pd.to_datetime(df['ipoDate'], format='%Y-%m-%d')
df['delistingDate'] = pd.to_datetime(df['delistingDate'], format='%Y-%m-%d')
df['assetType'] = df['assetType'].astype('category')
df['exchange'] = df['exchange'].astype('category')
df['name'] = df['name'].astype('string')

In [8]:
df.dtypes

symbol                   object
name             string[python]
exchange               category
assetType              category
ipoDate          datetime64[ns]
delistingDate    datetime64[ns]
status                   object
dtype: object

## Integração dos Dados

Para cada linha com nome nulo, vamos recuperar essa informação com a função do Yahoo Finance. Para as que não forem possíveis de recuperar, vamos excluir essas linhas:

In [9]:
df['name'] = df.apply(lambda x: get_finance_data_name(x.symbol) if pd.isna(x['name']) or x['name'] == '' else x.name, axis=1)

df.dropna(subset=['name', 'symbol'], inplace=True)
df.drop(df[df.name == ''].index, inplace=True)

df.shape


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AMEH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AMEH&crumb=P%2FStuqaRma7
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DFFN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DFFN&crumb=P%2FStuqaRma7
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NTEST-G?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=NTEST-G&crumb=P%2FStuqaRma7
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NTEST-J?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDoma

(11595, 7)

Vamos também excluir as duplicatas de nomes:

In [10]:
df.drop_duplicates(subset=['name'], inplace=True)
df.shape

(11595, 7)

Agora vamos integrar à nossa base de dados informações sobre os dividendos pagos pelas empresas e ETF's nos últimos 5 anos:

In [13]:
def get_dividends_by_period(period, symbol):
    try: 
        return yf.Ticker(symbol).history(period=period).Dividends.sum() # 1d, 1w, 1m, 3m, 6m, 5y, 10y, ytd, max
    except:
        return 0
    
get_dividends_by_period('5y', 'AAPL')

np.float64(4.58)

In [12]:
def get_history_by_period(period, symbol):
    try:
        return yf.Ticker(symbol).history(period=period)
    except:
        return pd.DataFrame()

get_history_by_period('5y', 'AAPL')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-03-19 00:00:00-04:00,60.036366,61.358966,58.876360,59.402973,271857200,0.0,0.0
2020-03-20 00:00:00-04:00,59.985399,61.113859,55.330817,55.631741,401693200,0.0,0.0
2020-03-23 00:00:00-04:00,55.350235,55.452160,51.595990,54.449894,336752800,0.0,0.0
2020-03-24 00:00:00-04:00,57.359606,60.109159,56.859688,59.912590,287531200,0.0,0.0
2020-03-25 00:00:00-04:00,60.851764,62.671857,59.286485,59.582554,303602000,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-12 00:00:00-04:00,220.139999,221.750000,214.910004,216.979996,62547500,0.0,0.0
2025-03-13 00:00:00-04:00,215.949997,216.839996,208.419998,209.679993,61368300,0.0,0.0
2025-03-14 00:00:00-04:00,211.250000,213.949997,209.580002,213.490005,60107600,0.0,0.0


## Redução e Validação dos Dados

Vamos extrair uma amostra aleatória para iniciar a comparação do desempenho nominal dos dividendos dos últimos 5 anos entre as bolsas.

In [14]:
sample_df = df.sample(n=25)
sample_df

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
6945,MSCI,6945,NYSE,Stock,2007-11-15,NaT,Active
7918,PBNV,7918,BATS,ETF,2024-05-22,NaT,Active
6198,LEGR,6198,NASDAQ,ETF,2018-01-25,NaT,Active
3823,FLC,3823,NYSE,ETF,2003-10-09,NaT,Active
8328,PRZO,8328,NASDAQ,Stock,2023-07-27,NaT,Active
8915,RNST,8915,NYSE,Stock,1992-04-24,NaT,Active
2224,COHN,2224,NYSE MKT,Stock,2004-05-06,NaT,Active
4125,FUTY,4125,NYSE ARCA,ETF,2013-10-24,NaT,Active
6075,KSTR,6075,NYSE ARCA,ETF,2021-01-27,NaT,Active
6836,MNTL,6836,NASDAQ,ETF,2024-01-23,NaT,Active


In [15]:
sample_df['dividends_last_5_years'] = sample_df.apply(lambda x: get_dividends_by_period('5y', x.symbol), axis=1)
sample_df

$PGRU: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
$RITM-P-C: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


,symbol,name,exchange,assetType,ipoDate,delistingDate,status,dividends_last_5_years
6945,MSCI,6945,NYSE,Stock,2007-11-15,NaT,Active,24.180000
7918,PBNV,7918,BATS,ETF,2024-05-22,NaT,Active,0.000000
6198,LEGR,6198,NASDAQ,ETF,2018-01-25,NaT,Active,4.202000
3823,FLC,3823,NYSE,ETF,2003-10-09,NaT,Active,6.609000
8328,PRZO,8328,NASDAQ,Stock,2023-07-27,NaT,Active,0.000000
8915,RNST,8915,NYSE,Stock,1992-04-24,NaT,Active,4.400000
2224,COHN,2224,NYSE MKT,Stock,2004-05-06,NaT,Active,4.250000
4125,FUTY,4125,NYSE ARCA,ETF,2013-10-24,NaT,Active,6.251000
6075,KSTR,6075,NYSE ARCA,ETF,2021-01-27,NaT,Active,0.000000
6836,MNTL,6836,NASDAQ,ETF,2024-01-23,NaT,Active,0.257000


Como a amostra aleatória trouxe muitas empresas novas, sem histórico de dividendos mínimo consistente para a nossa análise, vamos tentar refinar nossa amostra. Antes, vamos verificar a proporção de ativos por exchange e por tipo:

In [16]:
rate_exchange = df.exchange.value_counts(normalize=True)
rate_exchange

exchange
NASDAQ       0.447176
NYSE         0.263131
NYSE ARCA    0.192842
BATS         0.068650
NYSE MKT     0.028202
Name: proportion, dtype: float64

In [17]:
rate_asset = df.assetType.value_counts(normalize=True)
rate_asset

assetType
Stock    0.644761
ETF      0.355239
Name: proportion, dtype: float64

Para fins de simplificação, vamos deixar somente as exchanges mais conhecidas e verificar a proporção novamente:

In [18]:
df.drop(df[df.exchange == 'NYSE ARCA'].index, inplace=True)
df.drop(df[df.exchange == 'NYSE MKT'].index, inplace=True)
df.drop(df[df.exchange == 'BATS'].index, inplace=True)

In [19]:
rate_exchange = df.exchange.value_counts(normalize=True)
rate_exchange

exchange
NASDAQ       0.629553
NYSE         0.370447
BATS         0.000000
NYSE ARCA    0.000000
NYSE MKT     0.000000
Name: proportion, dtype: float64

Agora vamos extrair uma amostra obedecendo essas proporções:

In [23]:
exchange_sample = df.groupby('exchange').apply(lambda x: x.sample(int(10 * rate_exchange[x.name]))).droplevel('exchange')
exchange_sample

C:\Users\premier-improvement\AppData\Local\Temp\ipykernel_24328\2489681502.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  exchange_sample = df.groupby('exchange').apply(lambda x: x.sample(int(10 * rate_exchange[x.name]))).droplevel('exchange')
C:\Users\premier-improvement\AppData\Local\Temp\ipykernel_24328\2489681502.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exchange_sample = df.groupby('exchange').apply(lambda x: x.sample(int(10 * rate_exchange[x.name]))).droplevel('exchange')


,symbol,name,exchange,assetType,ipoDate,delistingDate,status
11299,WWD,11299,NASDAQ,Stock,1996-05-30,NaT,Active
4444,GLUE,4444,NASDAQ,Stock,2021-06-24,NaT,Active
9558,SMPL,9558,NASDAQ,Stock,2017-07-10,NaT,Active
8167,PLMIW,8167,NASDAQ,Stock,2021-05-06,NaT,Active
1389,BNDW,1389,NASDAQ,ETF,2018-09-06,NaT,Active
11477,XWEL,11477,NASDAQ,Stock,2018-01-05,NaT,Active
5936,KEN,5936,NYSE,Stock,2015-01-14,NaT,Active
457,AMG,457,NYSE,Stock,1997-11-21,NaT,Active
1407,BNS,1407,NYSE,Stock,1999-09-13,NaT,Active


In [24]:
asset_sample = df.groupby('assetType').apply(lambda x: x.sample(int(10 * rate_asset[x.name]))).droplevel('assetType')
asset_sample

C:\Users\premier-improvement\AppData\Local\Temp\ipykernel_24328\3961626324.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  asset_sample = df.groupby('assetType').apply(lambda x: x.sample(int(10 * rate_asset[x.name]))).droplevel('assetType')
C:\Users\premier-improvement\AppData\Local\Temp\ipykernel_24328\3961626324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  asset_sample = df.groupby('assetType').apply(lambda x: x.sample(int(10 * rate_asset[x.name]))).droplevel('assetType')


,symbol,name,exchange,assetType,ipoDate,delistingDate,status
10955,VONG,10955,NASDAQ,ETF,2010-09-22,NaT,Active
5580,IUSB,5580,NASDAQ,ETF,2014-06-12,NaT,Active
9176,SCDS,9176,NASDAQ,ETF,2024-08-08,NaT,Active
7865,PANL,7865,NASDAQ,Stock,2013-12-19,NaT,Active
11346,XELAP,11346,NASDAQ,Stock,2022-03-23,NaT,Active
7905,PBH,7905,NYSE,Stock,2005-02-10,NaT,Active
1441,BOTJ,1441,NASDAQ,Stock,2000-08-10,NaT,Active
9357,SHAK,9357,NYSE,Stock,2015-01-30,NaT,Active
11094,WASH,11094,NASDAQ,Stock,1995-08-18,NaT,Active


Para comparar dividendos, vamos adicionar as informações como anteriormente, em cada sample:

In [25]:
exchange_sample['dividends_last_5_years'] = exchange_sample.apply(lambda x: get_dividends_by_period('5y', x.symbol), axis=1)
exchange_sample


$PLMIW: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


,symbol,name,exchange,assetType,ipoDate,delistingDate,status,dividends_last_5_years
11299,WWD,11299,NASDAQ,Stock,1996-05-30,NaT,Active,3.815
4444,GLUE,4444,NASDAQ,Stock,2021-06-24,NaT,Active,0.000
9558,SMPL,9558,NASDAQ,Stock,2017-07-10,NaT,Active,0.000
8167,PLMIW,8167,NASDAQ,Stock,2021-05-06,NaT,Active,0.000
1389,BNDW,1389,NASDAQ,ETF,2018-09-06,NaT,Active,10.033
11477,XWEL,11477,NASDAQ,Stock,2018-01-05,NaT,Active,0.000
5936,KEN,5936,NYSE,Stock,2015-01-14,NaT,Active,24.430
457,AMG,457,NYSE,Stock,1997-11-21,NaT,Active,0.200
1407,BNS,1407,NYSE,Stock,1999-09-13,NaT,Active,14.219


In [26]:
asset_sample['dividends_last_5_years'] = asset_sample.apply(lambda x: get_dividends_by_period('5y', x.symbol), axis=1)
asset_sample

,symbol,name,exchange,assetType,ipoDate,delistingDate,status,dividends_last_5_years
10955,VONG,10955,NASDAQ,ETF,2010-09-22,NaT,Active,2.59425
5580,IUSB,5580,NASDAQ,ETF,2014-06-12,NaT,Active,6.99600
9176,SCDS,9176,NASDAQ,ETF,2024-08-08,NaT,Active,0.19000
7865,PANL,7865,NASDAQ,Stock,2013-12-19,NaT,Active,1.32500
11346,XELAP,11346,NASDAQ,Stock,2022-03-23,NaT,Active,0.83500
7905,PBH,7905,NYSE,Stock,2005-02-10,NaT,Active,0.00000
1441,BOTJ,1441,NASDAQ,Stock,2000-08-10,NaT,Active,1.56818
9357,SHAK,9357,NYSE,Stock,2015-01-30,NaT,Active,0.00000
11094,WASH,11094,NASDAQ,Stock,1995-08-18,NaT,Active,10.81000


Numa outra tentativa, vamos montar uma amostra considerando empresas listadas entre 5 e 10 anos:

In [33]:
five_year_df = df[df.ipoDate.dt.year.between(2014, 2024)]
five_year_df

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
1,AA,1,NYSE,Stock,2016-10-18,NaT,Active
5,AACT,5,NYSE,Stock,2023-06-12,NaT,Active
6,AACT-U,6,NYSE,Stock,2023-04-21,NaT,Active
7,AACT-WS,7,NYSE,Stock,2023-06-12,NaT,Active
8,AADI,8,NASDAQ,Stock,2017-08-08,NaT,Active
...,...,...,...,...,...,...,...
11618,ZVZZT,11618,NASDAQ,Stock,2017-09-22,NaT,Active
11620,ZWZZT,11620,NASDAQ,Stock,2017-09-22,NaT,Active
11621,ZXYZ-A,11621,NASDAQ,Stock,2016-01-19,NaT,Active
11623,ZYME,11623,NASDAQ,Stock,2017-04-28,NaT,Active


In [ ]:
five_year_rate = five_year_df.ipoDate.dt.year.value_counts(normalize=True)
five_year_rate

ipoDate
2021    0.251346
2022    0.116697
2023    0.107919
2020    0.104329
2024    0.083982
2019    0.065430
2018    0.061839
2017    0.060642
2014    0.058448
2016    0.048873
2015    0.040495
Name: proportion, dtype: float64

In [45]:
five_year_sample = five_year_df.groupby(five_year_df.ipoDate.dt.year).apply(lambda x: x.sample(int(25 * five_year_rate[x.name]))).droplevel('ipoDate')
five_year_sample

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
1655,BWLP,1655,NYSE,Stock,2014-01-28,NaT,Active
1859,CDTX,1859,NASDAQ,Stock,2015-04-15,NaT,Active
9241,SDG,9241,NASDAQ,ETF,2016-04-22,NaT,Active
7321,NODK,7321,NASDAQ,Stock,2017-03-16,NaT,Active
4834,HIVE,4834,NASDAQ,Stock,2018-01-12,NaT,Active
7372,NRUC,7372,NYSE,Stock,2019-05-08,NaT,Active
1569,BSY,1569,NASDAQ,Stock,2020-09-23,NaT,Active
6423,LU,6423,NYSE,Stock,2020-10-30,NaT,Active
10203,THCPW,10203,NASDAQ,Stock,2021-09-01,NaT,Active
4720,HAYW,4720,NYSE,Stock,2021-03-12,NaT,Active


In [46]:
five_year_sample['dividends_last_5_years'] = five_year_sample.apply(lambda x: get_dividends_by_period('5y', x.symbol), axis=1)
five_year_sample

$THCPW: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
$XFLT-P-A: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
$NNAGW: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


,symbol,name,exchange,assetType,ipoDate,delistingDate,status,dividends_last_5_years
1655,BWLP,1655,NYSE,Stock,2014-01-28,NaT,Active,7.840
1859,CDTX,1859,NASDAQ,Stock,2015-04-15,NaT,Active,0.000
9241,SDG,9241,NASDAQ,ETF,2016-04-22,NaT,Active,6.670
7321,NODK,7321,NASDAQ,Stock,2017-03-16,NaT,Active,0.000
4834,HIVE,4834,NASDAQ,Stock,2018-01-12,NaT,Active,0.000
7372,NRUC,7372,NYSE,Stock,2019-05-08,NaT,Active,6.536
1569,BSY,1569,NASDAQ,Stock,2020-09-23,NaT,Active,0.710
6423,LU,6423,NYSE,Stock,2020-10-30,NaT,Active,4.816
10203,THCPW,10203,NASDAQ,Stock,2021-09-01,NaT,Active,0.000
4720,HAYW,4720,NYSE,Stock,2021-03-12,NaT,Active,0.000
